# Usage
This is meant to streamline bulk-updating SOAR marketplace integrations. By default it will overwrite the existing ontology, so make sure to update the `override_ontology_mapping` option to change this if needed.

In [ ]:
# @title Update Marketplace Integrations
soar_url = "" # @param {"type":"string","placeholder":"https://my-soar.siemplify-soar.com"}
api_key = "" # @param {"type":"string","placeholder":"1111-2222-33333-4444444"}
override_ontology_mapping = True # @param {"type":"boolean"}
import requests
import json
import datetime

auth_header = {
        'content-type': 'application/json',
        'Appkey' : api_key
}

def main():

  # Main entry
  get_integrations()

def get_integrations():
  resp = requests.get(url=soar_url + "/api/external/v1/integrations/GetInstalledIntegrations?format=camel",
                       headers=auth_header
                      )
  mkt = resp.json()
  for i in mkt:
    # We're only interested in integrations from the Marketplace
    if i['isCertified'] == True:
      mkt_version = get_marketplace_version(i['identifier'])
      # If there is a version mismatch between local and Marketplace, we have an update
      if (str(mkt_version) != i['versionForDisplay']):
        print("Update found for: " + i['identifier'] + " (Marketplace Version: " + str(mkt_version) + " Installed Version: " + i['versionForDisplay'] + ")")
        update_integration(i['identifier'])


def get_marketplace_version(name):
  payload = {
    "integrationIdentifier": name,
  }
  resp = requests.post(url=soar_url + "/api/external/v1/store/GetIntegrationFullDetails?format=camel",
                       headers=auth_header, json=payload
                      )
  return resp.json()['version']

def update_integration(name):
  payload = {
    "identifier": name,
    "isCertified": True,
    "overrideMapping": override_ontology_mapping
  }
  resp = requests.post(url=soar_url + "/api/external/v1/store/DownloadAndInstallIntegrationFromLocalStore?format=camel&isStaging=false",
                       headers=auth_header, json=payload
                      )
  print(resp.text)


if __name__ == "__main__":
    main()